# Big Ideas in CS Final Project
# Team Baseball Pandas

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

dataBaseballFields = pd.read_csv('https://data.wprdc.org/dataset/5934b8d1-892f-4fd7-84c1-aa890d3ff529/resource/a61b6617-2f06-4745-8731-165139d41667/download/opendata-pubworks-field-listing-2015.csv')

# make dictionary
dataBaseballFields_dict = dict()

dataGreenSpaces = pd.read_csv('https://data.wprdc.org/dataset/cd9ae013-5b93-43d6-8793-bccf10f12c72/resource/b06fa202-ead7-491c-b8f6-2702dd35f8dd/download/opendata-pubworks-operations-green-spaces-2015.csv')

# make dictionary
dataGreenSpaces_dict = dict()